In [8]:
import re
import json
import requests
import csv
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import sqlite3
from category_encoders.one_hot import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import joblib
### ignore FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [49]:
# bad_idxs = []
# deleted_idxs = []

In [50]:
# for df_idx in tqdm(df.index[199:]):
#     try:
#         appid = str(df.loc[df_idx, "appid"])
#         url = f"http://store.steampowered.com/api/appdetails/?appids={appid}"
#         time.sleep(.5)
#         response = requests.get(url)
#         while response.json() is None:
#             time.sleep(10)
#             response = requests.get(url)
#         steam_app_data = response.json().get(appid, None)
#         if steam_app_data is None or steam_app_data["success"] is False or steam_app_data["data"]["type"] != "game":
#             deleted_idxs.append(df_idx)
#             continue

#         steam_app_data = steam_app_data["data"]
#         if "publishers" in steam_app_data and steam_app_data["publishers"]:
#             df.loc[df_idx, "publisher"] = steam_app_data["publishers"][0]
#         if "developers" in steam_app_data and steam_app_data["developers"]:
#             df.loc[df_idx, "developer"] = steam_app_data["developers"][0]
#         if ("is_free" in steam_app_data and steam_app_data["is_free"] == "true") or "price_overview" not in steam_app_data:
#             df.loc[df_idx, "initial_price"] = 0
#             df.loc[df_idx, "final_price"] = 0
#         else:
#             df.loc[df_idx, "initial_price"] = steam_app_data["price_overview"]["initial"]
#             df.loc[df_idx, "final_price"] = steam_app_data["price_overview"]["final"]
#         if "release_date" in steam_app_data and "date" in steam_app_data["release_date"]:
#             df.loc[df_idx, "release_date"] = steam_app_data["release_date"]["date"]
#         if "supported_languages" in steam_app_data and "korean" in steam_app_data["supported_languages"].lower():
#             df.loc[df_idx, "support_korean"] = True
#         else:
#             df.loc[df_idx, "support_korean"] = False
#         if "metacritic" in steam_app_data:
#             df.loc[df_idx, "metacritic"] = steam_app_data["metacritic"]["score"]

#         if "categories" in steam_app_data and steam_app_data["categories"]:
#             for category in steam_app_data["categories"]:
#                 if category["id"] not in steam_categories:
#                     steam_categories[category["id"]] = category["description"]
#                 df_categories.loc[len(df_categories)] = [appid, category["id"]]
#         if "genres" in steam_app_data and steam_app_data["genres"]:
#             for genre in steam_app_data["genres"]:
#                 if genre["id"] not in steam_genres:
#                     steam_genres[genre["id"]] = genre["description"]
#                 df_genres.loc[len(df_genres)] = [appid, genre["id"]]
#     except Exception as e:
#         bad_idxs.append(df_idx)
#         print(e)
        

In [29]:
data = dict()
for i in tqdm(range(60)):
    time.sleep(1)
    response = requests.get(f"http://steamspy.com/api.php?request=all&page={i}")
    
    if response.status_code == 200:
        # Parse the JSON data from the response
        data[i] = response.json()
    else:
        print("error!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [02:55<00:00,  2.92s/it]


In [40]:
df2 = pd.DataFrame()
for i in tqdm(data):
    for k, v in data[i].items():
        df2 = df2.append(v, ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [03:22<00:00,  3.37s/it]


In [41]:
len(df2)

59748

In [43]:
df2.duplicated().sum()

1835

In [44]:
df2

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,Dota 2,Valve,Valve,,1576161,326844,0,"200,000,000 .. 500,000,000",38051,1599,1060,1046,0,0,0,634918
1,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1206753,914944,0,"50,000,000 .. 100,000,000",21256,693,6623,185,0,0,0,460215
2,1063730,New World,Amazon Games,Amazon Games,,172856,75247,0,"50,000,000 .. 100,000,000",9354,1340,4100,1280,1999,3999,50,45237
3,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,6035588,796449,0,"50,000,000 .. 100,000,000",28944,695,6098,275,0,0,0,993889
4,440,Team Fortress 2,Valve,Valve,,868403,57762,0,"50,000,000 .. 100,000,000",8169,743,329,239,0,0,0,136514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59743,1069250,Eslander Alpha,Team Titan,Team Titan,,4,2,0,"0 .. 20,000",0,0,0,0,999,999,0,0
59744,1694380,Am Madness,James Stringer,Digital Crypt,,4,0,0,"0 .. 20,000",0,0,0,0,159,399,60,0
59745,40370,Emergency 2012,Quadriga Games,"Deep Silver, ValuSoft",,6,2,0,"0 .. 20,000",1192,0,1192,0,0,0,0,0
59746,1205160,Decadent Thinking,Hitori Games,Hitori Games,,7,3,0,"0 .. 20,000",0,0,0,0,1499,1499,0,0


In [112]:
df2[[("," in i) for i in df2["name"]]]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
181,6060,"Star Wars: Battlefront 2 (Classic, 2005)",Pandemic Studios,"Lucasfilm, LucasArts, Disney",,45851,2785,0,"2,000,000 .. 5,000,000",758,0,193,0,349,999,65,345
216,55150,"Warhammer 40,000: Space Marine - Anniversary E...",Relic Entertainment,SEGA,,19414,1713,0,"2,000,000 .. 5,000,000",441,284,320,371,1499,5999,75,410
338,239030,"Papers, Please",Lucas Pope,3909,,53857,1642,0,"2,000,000 .. 5,000,000",397,160,265,170,499,999,50,1196
387,15620,"Warhammer 40,000: Dawn of War II","Relic Entertainment, Feral Interactive (Mac/Li...","SEGA, Feral Interactive (Mac/Linux)",,9294,1127,0,"2,000,000 .. 5,000,000",925,0,415,0,499,1999,75,385
566,1058020,"STAR WARS Battlefront (Classic, 2004)",Pandemic Studios,"Lucasfilm, LucasArts, Disney",,3238,119,0,"1,000,000 .. 2,000,000",567,0,837,0,399,999,60,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58950,1670700,"Influence, Inc.",Curious Bird,Curious Bird,,12,2,0,"0 .. 20,000",0,0,0,0,599,1199,50,0
58951,1798740,"TEN - Ten Rooms, Ten Seconds",The Bworg,Ratalaika Games S.L.,,12,0,0,"0 .. 20,000",0,0,0,0,249,499,50,1
59051,2171290,Movavi Video Suite 2023 Steam Edition - An all...,Movavi,Movavi,,14,6,0,"0 .. 20,000",0,0,0,0,6649,9499,30,4
59335,1915450,"Death Damnation : Zombies, Ghosts and Vampires !",Ult.im,Ult.im,,25,10,0,"0 .. 20,000",0,0,0,0,839,1199,30,2


In [113]:
df_dropped = df2.drop_duplicates()
df_dropped

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,Dota 2,Valve,Valve,,1576161,326844,0,"200,000,000 .. 500,000,000",38051,1599,1060,1046,0,0,0,634918
1,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1206753,914944,0,"50,000,000 .. 100,000,000",21256,693,6623,185,0,0,0,460215
2,1063730,New World,Amazon Games,Amazon Games,,172856,75247,0,"50,000,000 .. 100,000,000",9354,1340,4100,1280,1999,3999,50,45237
3,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,6035588,796449,0,"50,000,000 .. 100,000,000",28944,695,6098,275,0,0,0,993889
4,440,Team Fortress 2,Valve,Valve,,868403,57762,0,"50,000,000 .. 100,000,000",8169,743,329,239,0,0,0,136514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59743,1069250,Eslander Alpha,Team Titan,Team Titan,,4,2,0,"0 .. 20,000",0,0,0,0,999,999,0,0
59744,1694380,Am Madness,James Stringer,Digital Crypt,,4,0,0,"0 .. 20,000",0,0,0,0,159,399,60,0
59745,40370,Emergency 2012,Quadriga Games,"Deep Silver, ValuSoft",,6,2,0,"0 .. 20,000",1192,0,1192,0,0,0,0,0
59746,1205160,Decadent Thinking,Hitori Games,Hitori Games,,7,3,0,"0 .. 20,000",0,0,0,0,1499,1499,0,0


In [114]:
df_dropped[df_dropped.duplicated(subset=["appid"])]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu


In [115]:
df_dropped = df_dropped.sort_values(by="appid").reset_index(drop=True)
df_dropped

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,10,Counter-Strike,Valve,Valve,,207952,5310,0,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514
1,20,Team Fortress Classic,Valve,Valve,,6000,952,0,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103
2,30,Day of Defeat,Valve,Valve,,5381,596,0,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141
3,40,Deathmatch Classic,Valve,Valve,,2020,459,0,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,,15488,790,0,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57908,2245700,Idle Game x100,PaulArt,PaulArt,,5,0,0,"0 .. 20,000",0,0,0,0,159,199,20,9
57909,2245840,ZAVOD,galesoozka,galesoozka,,11,1,0,"0 .. 20,000",0,0,0,0,339,399,15,0
57910,2249020,Mindbytes: Learn to Read Japanese,Ammonite Design Studios Ltd,Ammonite Design Studios Ltd,,0,1,0,"0 .. 20,000",0,0,0,0,239,299,20,0
57911,2251240,Police Car Simulator,"Yusuf Islam Seyhan, Samet Acar",Inspector Studios,,3,3,0,"0 .. 20,000",0,0,0,0,84,99,15,1


In [116]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57913 entries, 0 to 57912
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   appid            57913 non-null  int64 
 1   name             57913 non-null  object
 2   developer        57913 non-null  object
 3   publisher        57913 non-null  object
 4   score_rank       57913 non-null  object
 5   positive         57913 non-null  int64 
 6   negative         57913 non-null  int64 
 7   userscore        57913 non-null  int64 
 8   owners           57913 non-null  object
 9   average_forever  57913 non-null  int64 
 10  average_2weeks   57913 non-null  int64 
 11  median_forever   57913 non-null  int64 
 12  median_2weeks    57913 non-null  int64 
 13  price            57884 non-null  object
 14  initialprice     57891 non-null  object
 15  discount         57891 non-null  object
 16  ccu              57913 non-null  int64 
dtypes: int64(9), object(8)
memory u

In [117]:
df_dropped[(df_dropped["score_rank"]!="") | (df_dropped["userscore"]!=0)]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
554,22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,100,46753,2153,95,"500,000 .. 1,000,000",2602,464,1293,464,999,999,0,821
6201,371120,Discouraged Workers,YGGDRASIL STUDIO,YGGDRASIL STUDIO,98,61,38,59,"0 .. 20,000",0,0,0,0,299,1499,80,0
10648,502300,Heartomics: Lost Count,Heartomics,Heartomics,98,27,18,51,"0 .. 20,000",67,0,67,0,199,199,0,0
12621,555310,Satellite,"7DOTS, Rock Frog","7DOTS, Phoenix_co",99,44,9,82,"0 .. 20,000",185,0,242,0,149,299,50,0
12792,560000,Ladykiller in a Bind,Love Conquers All Games,Love Conquers All Games,100,170,8,95,"0 .. 20,000",148,0,221,0,999,1999,50,4
13854,588920,BADASS,Toothless T-Rex,大きくて強い,98,15,14,51,"0 .. 20,000",132,0,244,0,49,499,90,0
13996,592750,SPACE-FRIGHT,R I MAD,R I MAD,99,25,3,88,"0 .. 20,000",0,0,0,0,699,699,0,0
14376,603120,Happy Campers,Razzart Visual,Razzart Visual,98,28,31,46,"0 .. 20,000",0,0,0,0,299,999,70,0
15506,639780,Deep Space Waifu: FLAT JUSTICE,Neko Climax Studios,Neko Climax Studios,100,1449,67,95,"50,000 .. 100,000",170,0,158,0,149,299,50,2
16958,677730,Karmasutra,Top Hat Curios,Nutaku Publishing,99,13,6,60,"0 .. 20,000",0,0,0,0,1499,1499,0,0


In [118]:
df_dropped = df_dropped.drop(["score_rank", "userscore"], axis=1)
df_dropped

,appid,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,10,Counter-Strike,Valve,Valve,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514
1,20,Team Fortress Classic,Valve,Valve,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103
2,30,Day of Defeat,Valve,Valve,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141
3,40,Deathmatch Classic,Valve,Valve,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57908,2245700,Idle Game x100,PaulArt,PaulArt,5,0,"0 .. 20,000",0,0,0,0,159,199,20,9
57909,2245840,ZAVOD,galesoozka,galesoozka,11,1,"0 .. 20,000",0,0,0,0,339,399,15,0
57910,2249020,Mindbytes: Learn to Read Japanese,Ammonite Design Studios Ltd,Ammonite Design Studios Ltd,0,1,"0 .. 20,000",0,0,0,0,239,299,20,0
57911,2251240,Police Car Simulator,"Yusuf Islam Seyhan, Samet Acar",Inspector Studios,3,3,"0 .. 20,000",0,0,0,0,84,99,15,1


In [119]:
df_dropped["owners"].unique()

array(['10,000,000 .. 20,000,000', '5,000,000 .. 10,000,000',
       '1,000,000 .. 2,000,000', '2,000,000 .. 5,000,000',
       '20,000,000 .. 50,000,000', '50,000,000 .. 100,000,000',
       '200,000,000 .. 500,000,000', '0 .. 20,000', '20,000 .. 50,000',
       '500,000 .. 1,000,000', '100,000 .. 200,000', '200,000 .. 500,000',
       '50,000 .. 100,000'], dtype=object)

In [120]:
for col in ["price", "initialprice", "discount"]:
    df_dropped[col] = df_dropped[col].fillna(0).astype(int)

In [121]:
df_dropped

,appid,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,10,Counter-Strike,Valve,Valve,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514
1,20,Team Fortress Classic,Valve,Valve,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103
2,30,Day of Defeat,Valve,Valve,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141
3,40,Deathmatch Classic,Valve,Valve,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57908,2245700,Idle Game x100,PaulArt,PaulArt,5,0,"0 .. 20,000",0,0,0,0,159,199,20,9
57909,2245840,ZAVOD,galesoozka,galesoozka,11,1,"0 .. 20,000",0,0,0,0,339,399,15,0
57910,2249020,Mindbytes: Learn to Read Japanese,Ammonite Design Studios Ltd,Ammonite Design Studios Ltd,0,1,"0 .. 20,000",0,0,0,0,239,299,20,0
57911,2251240,Police Car Simulator,"Yusuf Islam Seyhan, Samet Acar",Inspector Studios,3,3,"0 .. 20,000",0,0,0,0,84,99,15,1


In [122]:
df_dropped.to_csv("games_all.csv", index=False)

In [128]:
df_candidates = df_dropped[(df_dropped["positive"] + df_dropped["negative"] >= 100) & \
                           (df_dropped["positive"]/(df_dropped["positive"] + df_dropped["negative"]) >= 0.5) & \
                           (df_dropped["ccu"] != 0)].set_index("appid")
df_candidates

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
appid,,,,,,,,,,,,,,
10,Counter-Strike,Valve,Valve,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514
20,Team Fortress Classic,Valve,Valve,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103
30,Day of Defeat,Valve,Valve,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141
40,Deathmatch Classic,Valve,Valve,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12
50,Half-Life: Opposing Force,Gearbox Software,Valve,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208920,Assassin's Creed Valhalla,Ubisoft Montreal,Ubisoft,2934,1941,"50,000 .. 100,000",545,371,475,565,1980,5999,67,9898
2209680,Beach Invasion 1944,AIx2 Games,AIx2 Games,281,57,"20,000 .. 50,000",0,0,0,0,999,999,0,15
2216360,Disaster Band,PRODUKTIVKELLER Studios,SunDust,137,20,"0 .. 20,000",13,13,13,13,699,699,0,68


In [153]:
df_candidates.drop(index=df_candidates[df_candidates["name"] == ""].index, inplace=True)

In [154]:
df_candidates[df_candidates.duplicated(keep=False, subset=["name"])].sort_values(by="name")

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,
9050,DOOM 3,id Software,id Software,5850,707,"500,000 .. 1,000,000",309,22,91,22,0,0,0,38,NaN
208200,DOOM 3,id Software,Bethesda Softworks,7705,1236,"1,000,000 .. 2,000,000",439,0,197,0,399,999,60,120,NaN
1173810,FINAL FANTASY V,Square Enix,Square Enix,964,154,"100,000 .. 200,000",201,0,353,0,1439,1799,20,195,NaN
382890,FINAL FANTASY V,Square Enix,Square Enix,1042,309,"100,000 .. 200,000",469,0,704,0,0,0,0,5,NaN
1173820,FINAL FANTASY VI,Square Enix,Square Enix,1970,222,"200,000 .. 500,000",89,0,85,0,1439,1799,20,312,NaN
382900,FINAL FANTASY VI,Square Enix,Square Enix,1987,437,"200,000 .. 500,000",446,0,704,0,0,0,0,23,NaN
22380,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,164978,6076,"5,000,000 .. 10,000,000",2742,157,891,214,249,999,75,5545,NaN
22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,46753,2153,"500,000 .. 1,000,000",2602,464,1293,464,999,999,0,821,NaN
245730,Flashback,,Ubisoft,276,194,"20,000 .. 50,000",0,0,0,0,249,999,75,3,NaN


In [155]:
dup_names = set(df_candidates[df_candidates.duplicated(subset=["name"])]["name"])
dup_names

{'DOOM 3',
 'FINAL FANTASY V',
 'FINAL FANTASY VI',
 'Fallout: New Vegas',
 'Flashback',
 'PICO PARK',
 'RUSH',
 'Scarlet Hollow',
 'Tiny Bunny',
 'War Trigger 3'}

In [52]:
# tags_overview.csv from steamspy.com
df_tags = pd.read_csv("tags_overview.csv")

In [72]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   #                    339 non-null    int64  
 1   Tag                  339 non-null    object 
 2   Games with this tag  339 non-null    int64  
 3   Votes for this tag   339 non-null    int32  
 4   Tag weight           339 non-null    int64  
 5   Price (median)       339 non-null    float64
 6   Userscore (median)   339 non-null    float64
 7   Owners (median)      339 non-null    int32  
 8   Playtime (median)    339 non-null    int64  
dtypes: float64(2), int32(2), int64(4), object(1)
memory usage: 21.3+ KB


In [79]:
df_tags = df_tags.sort_values(by="#").reset_index(drop=True)
df_tags

,id,tag_name,num_games,num_votes,tag_weight,med_price,med_score,med_owners,med_playtime
0,1,Indie,5156,277603,54,1.99,78.0,520000,63
1,2,Action,3878,442418,114,2.39,79.0,1210000,109
2,3,Adventure,3328,295101,89,2.49,79.0,1210000,124
3,4,Singleplayer,2539,269025,106,2.99,84.0,2240000,187
4,5,Casual,2462,90962,37,1.74,77.0,520000,29
...,...,...,...,...,...,...,...,...,...
334,335,Intentionally Awkward Controls,2,328,164,1.74,63.5,5520000,92
335,336,Cycling,2,47,24,9.36,83.0,1210000,166
336,337,Underground,2,77,39,0.99,70.0,2930000,35
337,338,Spelling,1,13,13,0.79,67.0,170000,0


In [56]:
df_tags["Votes for this tag"] = df_tags["Votes for this tag"].str.replace(",", "").astype(int)

In [63]:
df_tags["Price (median)"] = df_tags["Price (median)"].str.replace("$", "").astype(float)

In [65]:
df_tags["Userscore (median)"] = df_tags["Userscore (median)"].str.replace("%", "").astype(float)

In [66]:
df_tags["Owners (median)"] = df_tags["Owners (median)"].str.replace(",", "").astype(int)

In [67]:
df_tags["Playtime (median)"] = df_tags["Playtime (median)"].apply(lambda x: int(x[:2])*60 + int(x[-2:]))

In [77]:
df_tags.columns = ["id", "tag_name", "num_games", "num_votes", "tag_weight", "med_price", "med_score", "med_owners", "med_playtime"]

In [98]:
df_tags = df_tags.set_index("id")

In [99]:
df_tags

,tag_name,num_games,num_votes,tag_weight,med_price,med_score,med_owners,med_playtime
id,,,,,,,,
1,Indie,5156,277603,54,1.99,78.0,520000,63
2,Action,3878,442418,114,2.39,79.0,1210000,109
3,Adventure,3328,295101,89,2.49,79.0,1210000,124
4,Singleplayer,2539,269025,106,2.99,84.0,2240000,187
5,Casual,2462,90962,37,1.74,77.0,520000,29
...,...,...,...,...,...,...,...,...
335,Intentionally Awkward Controls,2,328,164,1.74,63.5,5520000,92
336,Cycling,2,47,24,9.36,83.0,1210000,166
337,Underground,2,77,39,0.99,70.0,2930000,35


In [27]:
df_tags = pd.read_csv("tags_overview_clean.csv", index_col=0)

In [28]:
tags_dict = dict()
for tag_id in tqdm(df_tags.index):
    tags_dict[df_tags.loc[tag_id, "tag_name"]] = tag_id

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 339/339 [00:00<00:00, 111606.68it/s]


In [31]:
import joblib
joblib.dump(tags_dict, "tags_to_id.pkl")

['tags_to_id.pkl']

In [32]:
import joblib
joblib.dump(tags_dict_rev, "id_to_tag.pkl")

['id_to_tag.pkl']

In [30]:
tags_dict_rev = dict()
for k, v in tags_dict.items():
    tags_dict_rev[v] = k

In [29]:
tags_dict

{'Indie': 1,
 'Action': 2,
 'Adventure': 3,
 'Singleplayer': 4,
 'Casual': 5,
 'Strategy': 6,
 'Simulation': 7,
 'RPG': 8,
 'Multiplayer': 9,
 'Great Soundtrack': 10,
 'Atmospheric': 11,
 '2D': 12,
 'Puzzle': 13,
 'Early Access': 14,
 'Open World': 15,
 'Story Rich': 16,
 'Co-op': 17,
 'Difficult': 18,
 'Shooter': 19,
 'Sci-fi': 20,
 'First-Person': 21,
 'Horror': 22,
 'VR': 23,
 'Anime': 24,
 'Pixel Graphics': 25,
 'Funny': 26,
 'Fantasy': 27,
 'Platformer': 28,
 'Female Protagonist': 29,
 'Free to Play': 30,
 'FPS': 31,
 'Survival': 32,
 'Gore': 33,
 'Violent': 34,
 'Sandbox': 35,
 'Retro': 36,
 'Arcade': 37,
 'Comedy': 38,
 'Classic': 39,
 'Third Person': 40,
 'Nudity': 41,
 'Massively Multiplayer': 42,
 'Exploration': 43,
 'Point & Click': 44,
 'Visual Novel': 45,
 'Turn-Based': 46,
 'Space': 47,
 'Sports': 48,
 'Rogue-like': 49,
 'Tactical': 50,
 'Cute': 51,
 'Racing': 52,
 'Psychological Horror': 53,
 'Online Co-Op': 54,
 'Zombies': 55,
 'Family Friendly': 56,
 'Sexual Content': 

In [78]:
df_tags.to_csv("tags_overview_clean.csv")

In [100]:
df_tags.describe()

,num_games,num_votes,tag_weight,med_price,med_score,med_owners,med_playtime
count,339.00000,339.000000,339.000000,339.000000,339.000000,3.390000e+02,339.000000
mean,202.79056,22370.333333,103.386431,3.029587,81.421829,2.359705e+06,176.085546
std,491.46963,51337.435104,84.945760,1.750841,5.789114,2.183794e+06,154.416620
min,1.00000,13.000000,1.000000,0.000000,58.000000,0.000000e+00,0.000000
25%,23.00000,1520.000000,46.000000,1.990000,78.500000,8.600000e+05,108.000000
50%,51.00000,4458.000000,75.000000,2.990000,82.000000,1.720000e+06,169.000000
75%,161.00000,17113.000000,132.500000,3.835000,85.000000,2.975000e+06,222.000000
max,5156.00000,442418.000000,454.000000,14.990000,97.000000,1.543000e+07,2324.000000


In [126]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome("chromedriver.exe")

for tag_id in tqdm(df_tags.index[::-1]):
    try:
        tag_name = df_tags.loc[tag_id, "tag_name"].strip().replace(" ", "+")
        driver.get(f"https://steamspy.com/tag/{tag_name}")

        # wait until someid is clickable
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'buttons-csv')))
        element.click()
    except Exception as e:
        print(tag_id, e)
driver.close()

C:\Users\line1\AppData\Local\Temp\ipykernel_6216\2670218372.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 339/339 [35:32<00:00,  6.29s/it]


In [221]:
df_candidates["release_date"] = False

In [222]:
df_game_tag_rel = pd.DataFrame(None, columns=["appid", "tagid"])
len(df_game_tag_rel)

0

In [223]:
bad_tag = []
game_name_set = set(df_candidates["name"])

In [224]:
for tag_id in tqdm(df_tags.index):
    tag_name = re.sub("[&']", "", df_tags.loc[tag_id, "tag_name"].strip())
    
    try:
        tmp_df = pd.read_csv(f"tag_game/{tag_name} - Tag Stats - SteamSpy - All the data and stats about Steam games.csv")
        for tmp_game_idx in tmp_df.index:
            tmp_game_name = tmp_df.loc[tmp_game_idx, "Game"]
            if tmp_game_name in game_name_set:
                for appid in df_candidates[df_candidates["name"]==tmp_game_name].index:
                    df_game_tag_rel.loc[len(df_game_tag_rel)] = [appid, tag_id]
                if any(df_candidates.loc[df_candidates["name"]==tmp_game_name, "release_date"]) is False:
                    if tmp_df.loc[tmp_game_idx, "Release date"] in ["Coming soon", "To be announced"]:
                        df_candidates.loc[df_candidates["name"]==tmp_game_name, "release_date"] = "tbd"
                        continue
                    tmp_game_date = pd.to_datetime(tmp_df.loc[tmp_game_idx, "Release date"])
                    df_candidates.loc[df_candidates["name"]==tmp_game_name, "release_date"] = tmp_game_date
    except Exception as e:
        bad_tag.append(tag_name)
        print(tag_name, e)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 339/339 [09:33<00:00,  1.69s/it]


In [232]:
need_date_idx = df_candidates[(df_candidates["release_date"] == False) | (df_candidates["release_date"] == "tbd")].index
need_info_idx = df_candidates[(df_candidates["release_date"] == False)].index

In [233]:
need_info_idx

Int64Index([   1313,    1630,    1670,   12390,   38400,  215710,  293220,
             294590,  296220,  339800,
            ...
            1487430, 1499560, 1506280, 1508570, 1576240, 1597280, 1602490,
            1660280, 1702830, 1801470],
           dtype='int64', name='appid', length=111)

In [246]:
df_game_tag_rel_copy = df_game_tag_rel.copy()

In [247]:
for appid in tqdm(need_info_idx):
    spy_url = f"http://steamspy.com/api.php?request=appdetails&appid={appid}"
    tags = requests.get(spy_url).json()["tags"].keys()
    for tag in tags:
        if tag in tags_dict:
            df_game_tag_rel.loc[len(df_game_tag_rel)] = [appid, tags_dict[tag]]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [02:48<00:00,  1.52s/it]


In [249]:
df_game_tag_rel

,appid,tagid
0,105600,1
1,1097150,1
2,252490,1
3,291550,1
4,304930,1
...,...,...
176152,1801470,36
176153,1801470,8
176154,1801470,129
176155,1801470,202


In [258]:
df_game_tag_rel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176157 entries, 0 to 176156
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   appid   176157 non-null  int64
 1   tagid   176157 non-null  int64
dtypes: int64(2)
memory usage: 4.0 MB


In [250]:
df_game_tag_rel_copy = df_game_tag_rel.copy()

In [253]:
need_date_idx_left = []
for appid in tqdm(need_date_idx):
    date_url = f"http://store.steampowered.com/api/appdetails/?appids={appid}"
    response = requests.get(date_url)
    if response.status_code == 200 and response.json()[str(appid)]["success"] and "release_date" in response.json()[str(appid)]["data"]:
        date = pd.to_datetime(response.json()[str(appid)]["data"]["release_date"]["date"])
        df_candidates.loc[appid, "release_date"] = date
    else:
        need_date_idx_left.append(appid)

 27%|███████████████████████████████████████████████▎                                                                                                                                 | 31/116 [00:19<00:50,  1.68it/s]

644560


 28%|██████████████████████████████████████████████████▎                                                                                                                              | 33/116 [00:20<00:49,  1.67it/s]

681820


 29%|███████████████████████████████████████████████████▉                                                                                                                             | 34/116 [00:21<00:49,  1.66it/s]

...

 93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊            | 108/116 [01:10<00:05,  1.38it/s]

1499560


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 115/116 [01:16<00:00,  1.19it/s]

1702830


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 116/116 [01:17<00:00,  1.50it/s]


In [254]:
need_date_idx_left = [644560, 681820, 681830, 681840, 723090, 928600, 1271190, 1499560, 1702830]

In [255]:
df_candidates.loc[need_date_idx_left,:]

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,
644560,Mirror,KAGAMI WORKS,Paradise Project,80419,2554,"2,000,000 .. 5,000,000",587,2,334,2,79,199,60,405,False
681820,(Chinese PaladinSword and Fairy 4),"Softstar Technology (Shanghai) Co., Ltd.","Beijing New Era Network Technology Co., Ltd.",492,108,"20,000 .. 50,000",1670,0,1871,0,599,599,0,371,False
681830,(Chinese PaladinSword and Fairy 5),"Softstar Technology (Beijing) Co.,Ltd","Beijing New Era Network Technology Co., Ltd.",76,36,"0 .. 20,000",1565,0,2609,0,599,599,0,14,False
681840,Chinese PaladinSword and Fairy 5 Prequel,"Softstar Technology (Beijing) Co.,Ltd","Beijing New Era Network Technology Co., Ltd.",90,31,"20,000 .. 50,000",1071,0,439,0,629,699,10,103,False
723090,Meltys Quest,Remtairy,"Remtairy, Kagura Games",446,10,"20,000 .. 50,000",576,0,798,0,1399,1999,30,37,False
928600,F1 2019,Codemasters,"Codemasters, Electronic Arts",19965,2263,"500,000 .. 1,000,000",2356,0,1742,0,0,0,0,148,False
1271190,Queen's Glory,"BananaKing, 台灣香蕉王",PlayMeow Games,431,178,"20,000 .. 50,000",34,0,34,0,379,999,62,1,False
1499560,LastCloudia,AIDIS,Boltrend Games,135,46,"100,000 .. 200,000",11213,0,18036,0,0,0,0,586,False
1702830,BLACKJACK and WAIFUS Hentai Version,KG/AM,KG/AM,83,17,"0 .. 20,000",0,0,0,0,69,99,30,4,False


In [256]:
df_candidates[(df_candidates["release_date"] == False) |(df_candidates["release_date"] == "tbd")]

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,
644560,Mirror,KAGAMI WORKS,Paradise Project,80419,2554,"2,000,000 .. 5,000,000",587,2,334,2,79,199,60,405,False
681820,(Chinese PaladinSword and Fairy 4),"Softstar Technology (Shanghai) Co., Ltd.","Beijing New Era Network Technology Co., Ltd.",492,108,"20,000 .. 50,000",1670,0,1871,0,599,599,0,371,False
681830,(Chinese PaladinSword and Fairy 5),"Softstar Technology (Beijing) Co.,Ltd","Beijing New Era Network Technology Co., Ltd.",76,36,"0 .. 20,000",1565,0,2609,0,599,599,0,14,False
681840,Chinese PaladinSword and Fairy 5 Prequel,"Softstar Technology (Beijing) Co.,Ltd","Beijing New Era Network Technology Co., Ltd.",90,31,"20,000 .. 50,000",1071,0,439,0,629,699,10,103,False
723090,Meltys Quest,Remtairy,"Remtairy, Kagura Games",446,10,"20,000 .. 50,000",576,0,798,0,1399,1999,30,37,False
928600,F1 2019,Codemasters,"Codemasters, Electronic Arts",19965,2263,"500,000 .. 1,000,000",2356,0,1742,0,0,0,0,148,False
1271190,Queen's Glory,"BananaKing, 台灣香蕉王",PlayMeow Games,431,178,"20,000 .. 50,000",34,0,34,0,379,999,62,1,False
1499560,LastCloudia,AIDIS,Boltrend Games,135,46,"100,000 .. 200,000",11213,0,18036,0,0,0,0,586,False
1702830,BLACKJACK and WAIFUS Hentai Version,KG/AM,KG/AM,83,17,"0 .. 20,000",0,0,0,0,69,99,30,4,False


In [257]:
# 한국 지역제한 게임 -> 제거..
df_candidates.drop(index=need_date_idx_left, inplace=True)
df_candidates[(df_candidates["release_date"] == False) |(df_candidates["release_date"] == "tbd")]

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,


In [259]:
df_candidates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12151 entries, 10 to 2231740
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             12151 non-null  object
 1   developer        12151 non-null  object
 2   publisher        12151 non-null  object
 3   positive         12151 non-null  int64 
 4   negative         12151 non-null  int64 
 5   owners           12151 non-null  object
 6   average_forever  12151 non-null  int64 
 7   average_2weeks   12151 non-null  int64 
 8   median_forever   12151 non-null  int64 
 9   median_2weeks    12151 non-null  int64 
 10  price            12151 non-null  int32 
 11  initialprice     12151 non-null  int32 
 12  discount         12151 non-null  int32 
 13  ccu              12151 non-null  int64 
 14  release_date     12144 non-null  object
dtypes: int32(3), int64(7), object(5)
memory usage: 1.3+ MB


In [265]:
df_candidates.to_csv("games.csv", quoting=csv.QUOTE_NONNUMERIC)

In [266]:
df_candidates

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,
10,Counter-Strike,Valve,Valve,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514,2000-11-01 00:00:00
20,Team Fortress Classic,Valve,Valve,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103,1999-04-01 00:00:00
30,Day of Defeat,Valve,Valve,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141,2003-05-01 00:00:00
40,Deathmatch Classic,Valve,Valve,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12,2001-06-01 00:00:00
50,Half-Life: Opposing Force,Gearbox Software,Valve,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218,1999-11-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208920,Assassin's Creed Valhalla,Ubisoft Montreal,Ubisoft,2934,1941,"50,000 .. 100,000",545,371,475,565,1980,5999,67,9898,2022-12-06 00:00:00
2209680,Beach Invasion 1944,AIx2 Games,AIx2 Games,281,57,"20,000 .. 50,000",0,0,0,0,999,999,0,15,2022-12-02 00:00:00
2216360,Disaster Band,PRODUKTIVKELLER Studios,SunDust,137,20,"0 .. 20,000",13,13,13,13,699,699,0,68,2022-12-20 00:00:00


In [272]:
df_game_tag_rel["tagid"].value_counts()[:100].index

Int64Index([  4,   1,   3,   2,   5,   9,   6,  12,   7,  11,   8,  16,  10,
             13,  24,  26,  27,  15,  17,  21,  43,  22,  25,  29,  18,  20,
             51,  14,  19,  35,  30,  40,  36,  38,  56,  32,  31,  37,  53,
             95,  34,  28,  41,  57,  33,  54,  44,  45,  87,  70,  82, 105,
             39,  63,  46,  60,  74, 102, 123, 137,  50,  66,  81,  76,  69,
             49,  59,  68,  72,  94,  78,  47,  80,  67,  48,  58, 108,  96,
             77,  42,  75,  64,  61,  23,  90,  73,  62, 110,  52,  99, 128,
            135, 165, 168,  84, 101, 145,  83, 129, 100],
           dtype='int64')

In [273]:
tags_dict_rev = dict()
for k, v in tags_dict.items():
    tags_dict_rev[v] = k

In [288]:
df_game_tag_rel.to_csv("game_tag_rel.csv", index=False)

In [278]:
df_candidates.sort_values(by="average_forever", ascending=False).iloc[:60]

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,
353330,Love at First Sight,Creepy Cute,Sekai Project,1190,52,"20,000 .. 50,000",1303016,11259,1303040,11259,249,999,75,5,2015-05-18 00:00:00
1379130,League of Angels-Heaven's Fury,"CHENGDU DREAMCALLER TECHNOLOGY CO.,LTD",YOUZU SINGAPORE PTE LTD,350,261,"50,000 .. 100,000",341839,0,341839,0,0,0,0,111,2020-09-03 00:00:00
1283970,YoloMouse,Dragonrise Games,Dragonrise Games,1063,76,"200,000 .. 500,000",172613,9741,238200,9741,399,399,0,5273,2020-05-01 00:00:00
319080,Disney Princess: My Fairytale Adventure,High Impact Games,Disney,180,24,"0 .. 20,000",75627,0,151213,0,599,1999,70,16,2012-05-25 00:00:00
702320,March of Empires,Gameloft,Gameloft,843,421,"200,000 .. 500,000",72800,0,109187,0,0,0,0,1048,2017-12-11 00:00:00
1071100,Boom 3D,Global Delight Technologies Pvt. Ltd.,Global Delight Technologies Pvt. Ltd.,474,56,"50,000 .. 100,000",61551,0,61551,0,1359,1699,20,129,2019-08-26 00:00:00
458750,FireAlpaca SE,PGN Inc,PGN Inc,113,9,"20,000 .. 50,000",49189,0,49189,0,2799,3999,30,75,2016-05-22 00:00:00
628050,Romance of the Three Kingdoms IX with Power Up...,"KOEI TECMO GAMES CO., LTD.","KOEI TECMO GAMES CO., LTD.",307,57,"50,000 .. 100,000",42238,0,42238,0,2999,2999,0,190,2017-10-17 00:00:00
524660,RutonyChat,Rutony Studio,Rutony Studio,567,155,"100,000 .. 200,000",41716,0,41716,0,1499,1499,0,639,2016-09-19 00:00:00


In [283]:
df_dropped[df_dropped["name"]==""]

,appid,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
475,17760,,,,0,0,"0 .. 20,000",0,0,0,0,0,0,0,0
9372,460250,,Jeroen Wimmers,Jeroen Wimmers,100,10,"0 .. 20,000",0,0,0,0,0,0,0,1
10822,506630,,,EGAMER,1,0,"0 .. 20,000",0,0,0,0,0,0,0,0
13432,576960,,Nuclear Tales,,110,26,"20,000 .. 50,000",267,0,301,0,299,1499,80,3
21926,806160,,Paleno Games,Paleno Games,6,17,"20,000 .. 50,000",0,0,0,0,49,99,51,0
21928,806220,,"Jochen Heizmann, Emmanuel Henné, Intermediaware",Asylum Square,23,13,"0 .. 20,000",0,0,0,0,0,0,0,0
27624,965340,,2nd Studio,2nd Studio,55,7,"0 .. 20,000",0,0,0,0,199,199,0,0
31346,1071920,,Ultimo Games,Ultimo Games,19,0,"0 .. 20,000",0,0,0,0,399,399,0,0
32861,1116910,,,,12,16,"0 .. 20,000",0,0,0,0,699,699,0,0


In [285]:
df_all = df_dropped.drop(index=df_dropped[df_dropped["name"]==""].index).set_index("appid")
df_all

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
appid,,,,,,,,,,,,,,
10,Counter-Strike,Valve,Valve,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514
20,Team Fortress Classic,Valve,Valve,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103
30,Day of Defeat,Valve,Valve,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141
40,Deathmatch Classic,Valve,Valve,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12
50,Half-Life: Opposing Force,Gearbox Software,Valve,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245700,Idle Game x100,PaulArt,PaulArt,5,0,"0 .. 20,000",0,0,0,0,159,199,20,9
2245840,ZAVOD,galesoozka,galesoozka,11,1,"0 .. 20,000",0,0,0,0,339,399,15,0
2249020,Mindbytes: Learn to Read Japanese,Ammonite Design Studios Ltd,Ammonite Design Studios Ltd,0,1,"0 .. 20,000",0,0,0,0,239,299,20,0


In [290]:
df_game_tag_rel_all = pd.DataFrame(None, columns=["appid", "tagid"])
len(df_game_tag_rel_all)

0

In [291]:
bad_tag = []
game_name_set_all = set(df_all["name"])

In [292]:
for tag_id in tqdm(df_tags.index):
    tag_name = re.sub("[&']", "", df_tags.loc[tag_id, "tag_name"].strip())
    
    try:
        tmp_df = pd.read_csv(f"tag_game/{tag_name} - Tag Stats - SteamSpy - All the data and stats about Steam games.csv")
        for tmp_game_idx in tmp_df.index:
            tmp_game_name = tmp_df.loc[tmp_game_idx, "Game"]
            if tmp_game_name in game_name_set_all:
                for appid in df_all[df_all["name"]==tmp_game_name].index:
                    df_game_tag_rel_all.loc[len(df_game_tag_rel_all)] = [appid, tag_id]
    except Exception as e:
        bad_tag.append(tag_name)
        print(tag_name, e)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 339/339 [1:45:39<00:00, 18.70s/it]


In [293]:
df_game_tag_rel_all

,appid,tagid
0,105600,1
1,1097150,1
2,252490,1
3,856300,1
4,291550,1
...,...,...
647107,485900,339
647108,983050,339
647109,1052030,339
647110,1260310,339


In [294]:
df_game_tag_rel_all.to_csv("game_tag_rel_all.csv", index=False)

In [299]:
tmp_set = set(df_game_tag_rel_all["appid"])
tmp_set2 = set(df_all.index)
need_info_idx = list(tmp_set2.difference(tmp_set))

In [300]:
df_game_tag_rel_all_copy = df_game_tag_rel_all.copy()

In [301]:
df_game_tag_rel_all_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 647112 entries, 0 to 647111
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   appid   647112 non-null  int64
 1   tagid   647112 non-null  int64
dtypes: int64(2)
memory usage: 14.8 MB


In [304]:
df_game_tag_rel_all = df_game_tag_rel_all_copy.copy()

In [303]:
need_info_idx

[1003520,
 47110,
 630790,
 704520,
 1314830,
 958480,
 1015830,
 1509400,
 1013790,
 933920,
 555040,
 1222690,
 743460,
 1667110,
 845870,
 981040,
 213050,
 942140,
 974920,
 888910,
 12370,
 596050,
 1017940,
 712790,
 12380,
 12390,
 983150,
 501870,
 1394800,
 872560,
 671860,
 790650,
 616570,
 1202300,
 557180,
 866430,
 1753220,
 956550,
 833670,
 1755270,
 780430,
 948370,
 723090,
 12440,
 975000,
 874650,
 878750,
 483490,
 700580,
 981160,
 975020,
 616620,
 405680,
 846000,
 764080,
 553140,
 1271990,
 950460,
 1560770,
 848070,
 983240,
 729290,
 989390,
 1157340,
 1190110,
 1548510,
 803040,
 596200,
 4330,
 16620,
 653550,
 633080,
 985340,
 749820,
 823550,
 743680,
 887040,
 938250,
 827660,
 1136910,
 1466640,
 346390,
 901399,
 1313050,
 1210650,
 833820,
 508190,
 450850,
 813350,
 1653030,
 497960,
 762150,
 420140,
 461100,
 555310,
 362800,
 1198390,
 801080,
 391480,
 928060,
 1624380,
 254270,
 440640,
 1325380,
 1096010,
 1204560,
 694610,
 792920,
 29017,
 

In [305]:
for appid in tqdm(need_info_idx):
    spy_url = f"http://steamspy.com/api.php?request=appdetails&appid={appid}"
    response = requests.get(spy_url)
    if response.status_code == 200 and "tags" in response.json() and response.json()["tags"]:
        tags = requests.get(spy_url).json()["tags"].keys()
        for tag in tags:
            if tag in tags_dict:
                df_game_tag_rel_all.loc[len(df_game_tag_rel_all)] = [appid, tags_dict[tag]]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 705/705 [26:43<00:00,  2.27s/it]


In [306]:
df_game_tag_rel_all

,appid,tagid
0,105600,1
1,1097150,1
2,252490,1
3,856300,1
4,291550,1
...,...,...
651409,1148920,5
651410,1148920,1
651411,1148920,3
651412,1148920,57


In [319]:
tmp_set = set(df_game_tag_rel_all["appid"])
tmp_set2 = set(df_all.index)
need_info_idx = list(tmp_set2.difference(tmp_set))
len(need_info_idx)

324

In [307]:
df_game_tag_rel_all.to_csv("game_tag_rel_all.csv", index=False)

In [363]:
ohe = OneHotEncoder(cols=["tagid"], use_cat_names=True)
genre_onehot = ohe.fit_transform(df_game_tag_rel_all)
genre_onehot

,appid,tagid_1.0,tagid_2.0,tagid_3.0,tagid_4.0,tagid_5.0,tagid_6.0,tagid_7.0,tagid_8.0,tagid_9.0,...,tagid_330.0,tagid_331.0,tagid_332.0,tagid_333.0,tagid_334.0,tagid_335.0,tagid_336.0,tagid_337.0,tagid_338.0,tagid_339.0
0,105600,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1097150,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,252490,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,856300,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,291550,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651409,1148920,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
651410,1148920,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
651411,1148920,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
651412,1148920,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [364]:
genre_onehot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651414 entries, 0 to 651413
Columns: 336 entries, appid to tagid_339.0
dtypes: int64(336)
memory usage: 1.6 GB


In [365]:
game_genre_vector_all = genre_onehot.groupby("appid").any()

In [366]:
no_tags = []
for i in range(1, 340):
    if f"tagid_{i}.0" not in game_genre_vector_all.columns:
        no_tags.append(i)
no_tags

[278, 289, 298, 299]

In [367]:
for i in no_tags:
    game_genre_vector_all[f"tagid_{i}.0"] = False

In [368]:
game_genre_vector_all.columns = [i[:-2] for i in game_genre_vector_all.columns]

In [369]:
game_genre_vector_all = game_genre_vector_all[[f"tagid_{i}" for i in range(1, 340)]]

In [370]:
game_genre_vector_all

,tagid_1,tagid_2,tagid_3,tagid_4,tagid_5,tagid_6,tagid_7,tagid_8,tagid_9,tagid_10,...,tagid_330,tagid_331,tagid_332,tagid_333,tagid_334,tagid_335,tagid_336,tagid_337,tagid_338,tagid_339
appid,,,,,,,,,,,,,,,,,,,,,
10,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
20,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
30,False,True,False,True,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
40,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
50,False,True,True,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245700,False,False,False,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2245840,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2249020,True,False,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [371]:
game_genre_vector_all.to_csv("game_genre_vector_all.csv")

In [372]:
ohe = OneHotEncoder(cols=["tagid"], use_cat_names=True)
genre_onehot_c = ohe.fit_transform(df_game_tag_rel)
genre_onehot_c

,appid,tagid_1.0,tagid_2.0,tagid_3.0,tagid_4.0,tagid_5.0,tagid_6.0,tagid_7.0,tagid_8.0,tagid_9.0,...,tagid_330.0,tagid_331.0,tagid_332.0,tagid_333.0,tagid_334.0,tagid_335.0,tagid_336.0,tagid_337.0,tagid_338.0,tagid_339.0
0,105600,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1097150,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,252490,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,291550,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,304930,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176152,1801470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176153,1801470,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
176154,1801470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176155,1801470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [373]:
game_genre_vector = genre_onehot_c.groupby("appid").any()

In [374]:
no_tags = []
for i in range(1, 340):
    if f"tagid_{i}.0" not in game_genre_vector.columns:
        no_tags.append(i)
no_tags

[278, 289, 298, 299]

In [375]:
for i in no_tags:
    game_genre_vector[f"tagid_{i}.0"] = False

In [376]:
game_genre_vector.columns = [i[:-2] for i in game_genre_vector.columns]
game_genre_vector = game_genre_vector[[f"tagid_{i}" for i in range(1, 340)]]
game_genre_vector

,tagid_1,tagid_2,tagid_3,tagid_4,tagid_5,tagid_6,tagid_7,tagid_8,tagid_9,tagid_10,...,tagid_330,tagid_331,tagid_332,tagid_333,tagid_334,tagid_335,tagid_336,tagid_337,tagid_338,tagid_339
appid,,,,,,,,,,,,,,,,,,,,,
10,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
20,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
30,False,True,False,True,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
40,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
50,False,True,True,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208920,False,True,True,True,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2209680,False,True,False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2216360,True,False,False,True,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [377]:
different_vector_appid = []
for appid in tqdm(game_genre_vector.index):
    if all(game_genre_vector.loc[appid] == game_genre_vector_all.loc[appid]) == False:
        different_vector_appid.append(appid)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12160/12160 [00:01<00:00, 6530.12it/s]


In [378]:
# 죠아요 오홍홍
different_vector_appid

[]

In [379]:
game_genre_vector.to_csv("game_genre_vector.csv")

In [380]:
df_candidates

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,
10,Counter-Strike,Valve,Valve,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514,2000-11-01 00:00:00
20,Team Fortress Classic,Valve,Valve,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103,1999-04-01 00:00:00
30,Day of Defeat,Valve,Valve,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141,2003-05-01 00:00:00
40,Deathmatch Classic,Valve,Valve,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12,2001-06-01 00:00:00
50,Half-Life: Opposing Force,Gearbox Software,Valve,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218,1999-11-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208920,Assassin's Creed Valhalla,Ubisoft Montreal,Ubisoft,2934,1941,"50,000 .. 100,000",545,371,475,565,1980,5999,67,9898,2022-12-06 00:00:00
2209680,Beach Invasion 1944,AIx2 Games,AIx2 Games,281,57,"20,000 .. 50,000",0,0,0,0,999,999,0,15,2022-12-02 00:00:00
2216360,Disaster Band,PRODUKTIVKELLER Studios,SunDust,137,20,"0 .. 20,000",13,13,13,13,699,699,0,68,2022-12-20 00:00:00


In [383]:
game_genre_vector.drop(index=need_date_idx_left, inplace=True)

In [385]:
df_candidates.drop(columns=["developer", "publisher", "price", "initialprice", "discount"])

,name,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,ccu,release_date
appid,,,,,,,,,,
10,Counter-Strike,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,15514,2000-11-01 00:00:00
20,Team Fortress Classic,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,103,1999-04-01 00:00:00
30,Day of Defeat,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,141,2003-05-01 00:00:00
40,Deathmatch Classic,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,12,2001-06-01 00:00:00
50,Half-Life: Opposing Force,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,218,1999-11-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
2208920,Assassin's Creed Valhalla,2934,1941,"50,000 .. 100,000",545,371,475,565,9898,2022-12-06 00:00:00
2209680,Beach Invasion 1944,281,57,"20,000 .. 50,000",0,0,0,0,15,2022-12-02 00:00:00
2216360,Disaster Band,137,20,"0 .. 20,000",13,13,13,13,68,2022-12-20 00:00:00


In [2]:
game_genre_vector = pd.read_csv("game_genre_vector.csv", index_col=0)

In [3]:
tag_sim = cosine_similarity(game_genre_vector, game_genre_vector, dense_output=False)

In [4]:
tag_sim

array([[1.        , 0.5       , 0.54073807, ..., 0.06804138, 0.12598816,
        0.05892557],
       [0.5       , 1.        , 0.54073807, ..., 0.20412415, 0.12598816,
        0.        ],
       [0.54073807, 0.54073807, 1.        , ..., 0.13245324, 0.1839418 ,
        0.        ],
       ...,
       [0.06804138, 0.20412415, 0.13245324, ..., 1.        , 0.15430335,
        0.07216878],
       [0.12598816, 0.12598816, 0.1839418 , ..., 0.15430335, 1.        ,
        0.13363062],
       [0.05892557, 0.        , 0.        , ..., 0.07216878, 0.13363062,
        1.        ]])

In [5]:
joblib.dump(tag_sim, "tag_sum.pkl")

['tag_sum.pkl']

In [6]:
tag_sim_idx = np.array([i[::-1] for i in tag_sim.argsort()])

In [7]:
joblib.dump(tag_sim_idx, "tag_sim_idx.pkl")

['tag_sim_idx.pkl']

In [8]:
df = pd.DataFrame(tag_sim_idx)

In [9]:
df.to_csv("tag_sim_idx.csv", index=False)

In [10]:
tag_sim_idx

array([[    0,    10,     7, ...,  7508,  7517,  6079],
       [    1,     3,  3981, ...,  6225,  6226,  6079],
       [    2,   304,   299, ...,  6813,  6806, 12159],
       ...,
       [12157,  8540,  6518, ...,  4222,   809,  1765],
       [12158,  2735, 10573, ...,  9398,  3348,  5350],
       [12159,  3489, 11702, ...,  8378,  3016,  8672]], dtype=int64)

In [11]:
tag_sim_idx[0, :20]

array([    0,    10,     7,    25,    26,  5355,   488,  6198,  1128,
         493,   304, 10304,   358,   427,   492,   292,   299,     2,
        6995,  3502], dtype=int64)

In [13]:
df_candidates = pd.read_csv("games.csv", index_col=0)
df_candidates

,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,release_date
appid,,,,,,,,,,,,,,,
10,Counter-Strike,Valve,Valve,207952,5310,"10,000,000 .. 20,000,000",7677,83,209,54,99,999,90,15514,2000-11-01 00:00:00
20,Team Fortress Classic,Valve,Valve,6000,952,"5,000,000 .. 10,000,000",314,39,15,39,49,499,90,103,1999-04-01 00:00:00
30,Day of Defeat,Valve,Valve,5381,596,"5,000,000 .. 10,000,000",630,0,24,0,49,499,90,141,2003-05-01 00:00:00
40,Deathmatch Classic,Valve,Valve,2020,459,"5,000,000 .. 10,000,000",34,0,7,0,49,499,90,12,2001-06-01 00:00:00
50,Half-Life: Opposing Force,Gearbox Software,Valve,15488,790,"5,000,000 .. 10,000,000",455,0,125,0,49,499,90,218,1999-11-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208920,Assassin's Creed Valhalla,Ubisoft Montreal,Ubisoft,2934,1941,"50,000 .. 100,000",545,371,475,565,1980,5999,67,9898,2022-12-06 00:00:00
2209680,Beach Invasion 1944,AIx2 Games,AIx2 Games,281,57,"20,000 .. 50,000",0,0,0,0,999,999,0,15,2022-12-02 00:00:00
2216360,Disaster Band,PRODUKTIVKELLER Studios,SunDust,137,20,"0 .. 20,000",13,13,13,13,699,699,0,68,2022-12-20 00:00:00


In [15]:
df_candidates.columns

Index(['name', 'developer', 'publisher', 'positive', 'negative', 'owners',
       'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks',
       'price', 'initialprice', 'discount', 'ccu', 'release_date'],
      dtype='object')

In [ ]:
df_candidates.drop(columns=['developer', 'publisher','price', 'initialprice', 'discount'], inplace=True)
df_candidates["total_reviews"] = df_candidates["positive"] + df_candidates["negative"]

In [18]:
df_candidates["rating"] = (df_candidates["positive"] / df_candidates["total_reviews"] * 100)

In [23]:
C = df_candidates["rating"].mean()
m = df_candidates["total_reviews"].quantile(0.3)

In [24]:
df_candidates["weighted_rating"] = (((df_candidates["total_reviews"]/(df_candidates["total_reviews"] + m)) * df_candidates["rating"]) + \
                                   ((m / (m + df_candidates["total_reviews"])) * C))

In [28]:
df_final = df_candidates.drop(columns=["owners", "average_forever", 'average_2weeks', 'median_forever', 'median_2weeks','positive', 'negative', "total_reviews", "rating"])

In [35]:
df_final["release_date"] = df_final["release_date"].astype('datetime64')

In [39]:
df_final = df_final[["name", "release_date", "ccu", "weighted_rating"]]

In [40]:
df_final

,name,release_date,ccu,weighted_rating
appid,,,,
10,Counter-Strike,2000-11-01,15514,97.486678
20,Team Fortress Classic,1999-04-01,103,86.125120
30,Day of Defeat,2003-05-01,141,89.625277
40,Deathmatch Classic,2001-06-01,12,81.580127
50,Half-Life: Opposing Force,1999-11-01,218,94.892334
...,...,...,...,...
2208920,Assassin's Creed Valhalla,2022-12-06,9898,61.582820
2209680,Beach Invasion 1944,2022-12-02,15,82.724132
2216360,Disaster Band,2022-12-20,68,83.903285


In [41]:
df_final.to_csv("games_final.csv", quoting=csv.QUOTE_NONNUMERIC)

In [63]:
df_final["release_date"] = df_final["release_date"].astype(str)
df_final

,name,release_date,ccu,weighted_rating
appid,,,,
10,Counter-Strike,2000-11-01 00:00:00,15514,97.486678
20,Team Fortress Classic,1999-04-01 00:00:00,103,86.125120
30,Day of Defeat,2003-05-01 00:00:00,141,89.625277
40,Deathmatch Classic,2001-06-01 00:00:00,12,81.580127
50,Half-Life: Opposing Force,1999-11-01 00:00:00,218,94.892334
...,...,...,...,...
2208920,Assassin's Creed Valhalla,2022-12-06 00:00:00,9898,61.582820
2209680,Beach Invasion 1944,2022-12-02 00:00:00,15,82.724132
2216360,Disaster Band,2022-12-20 00:00:00,68,83.903285


In [51]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12151 entries, 10 to 2231740
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12151 non-null  object 
 1   release_date     12144 non-null  object 
 2   ccu              12151 non-null  int64  
 3   weighted_rating  12151 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 474.6+ KB


In [57]:
from datetime import datetime
type(datetime.now())

datetime.datetime

In [64]:
games = df_final.reset_index().to_dict("records")
games

[{'appid': 10,
  'name': 'Counter-Strike',
  'release_date': '2000-11-01 00:00:00',
  'ccu': 15514,
  'weighted_rating': 97.48667779236044},
 {'appid': 20,
  'name': 'Team Fortress Classic',
  'release_date': '1999-04-01 00:00:00',
  'ccu': 103,
  'weighted_rating': 86.12511967422832},
 {'appid': 30,
  'name': 'Day of Defeat',
  'release_date': '2003-05-01 00:00:00',
  'ccu': 141,
  'weighted_rating': 89.62527693435717},
 {'appid': 40,
  'name': 'Deathmatch Classic',
  'release_date': '2001-06-01 00:00:00',
  'ccu': 12,
  'weighted_rating': 81.58012690457846},
 {'appid': 50,
  'name': 'Half-Life: Opposing Force',
  'release_date': '1999-11-01 00:00:00',
  'ccu': 218,
  'weighted_rating': 94.89233433781274},
 {'appid': 60,
  'name': 'Ricochet',
  'release_date': '2022-01-11 00:00:00',
  'ccu': 11,
  'weighted_rating': 81.85055849364757},
 {'appid': 70,
  'name': 'Half-Life',
  'release_date': '1998-11-08 00:00:00',
  'ccu': 1640,
  'weighted_rating': 96.52729545214542},
 {'appid': 80,
 

In [21]:
import sqlite3

In [4]:
conn = sqlite3.connect("flask_app.db")

In [5]:
cur = conn.cursor()

In [65]:
cur.executemany(
    """
    INSERT INTO game
    VALUES (:appid, :name, :release_date, :ccu, :weighted_rating);
    """,
    games
)

In [66]:
conn.commit()

In [6]:
game_genre_vector_all = pd.read_csv("game_genre_vector_all.csv")
game_genre_vector_all

,appid,tagid_1,tagid_2,tagid_3,tagid_4,tagid_5,tagid_6,tagid_7,tagid_8,tagid_9,...,tagid_330,tagid_331,tagid_332,tagid_333,tagid_334,tagid_335,tagid_336,tagid_337,tagid_338,tagid_339
0,10,False,True,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,20,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,30,False,True,False,True,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,40,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,50,False,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57575,2245700,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
57576,2245840,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
57577,2249020,True,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
57578,2251240,True,True,False,True,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False


In [2]:
game_genre_vector = pd.read_csv("game_genre_vector.csv", index_col=0)
game_genre_vector

,tagid_1,tagid_2,tagid_3,tagid_4,tagid_5,tagid_6,tagid_7,tagid_8,tagid_9,tagid_10,...,tagid_330,tagid_331,tagid_332,tagid_333,tagid_334,tagid_335,tagid_336,tagid_337,tagid_338,tagid_339
appid,,,,,,,,,,,,,,,,,,,,,
10,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
20,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
30,False,True,False,True,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
40,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
50,False,True,True,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208920,False,True,True,True,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
2209680,False,True,False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2216360,True,False,False,True,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [3]:
c = game_genre_vector.index.isin([10, 20, 4, 12342, 64480])
c

array([ True,  True, False, ..., False, False, False])

In [14]:
b = game_genre_vector[~game_genre_vector.index.isin([10, 20, 4, 12342, 64480])]

In [9]:
phs = [f":tagid_{i}" for i in range(1, 340)]
cur.executemany(
    f"""
    INSERT INTO game_genre_vector
    VALUES (:appid, {",".join(phs)});
    """,
    game_genre_vector.to_dict("records")
)

In [10]:
conn.commit()

In [11]:
cur.executemany(
    f"""
    INSERT INTO game_genre_vector_all
    VALUES (:appid, {",".join(phs)});
    """,
    game_genre_vector_all.to_dict("records")
)

In [12]:
conn.commit()

In [33]:
id_to_tag = joblib.load("id_to_tag.pkl")
tag_to_id = joblib.load("tag_to_id.pkl")

In [16]:
a = np.array([1 if i%2 else 0 for i in range(1, 340)]).reshape(1, -1)

In [17]:
cosine_similarity(a, b)

array([[0.19354922, 0.17822656, 0.18864844, ..., 0.13284223, 0.14348601,
        0.2300895 ]])

In [2]:
from flask_app.views.analysis_views import find_sim_game, get_feature_vector_from_db, get_feature_vector_from_web

In [15]:
import sqlite3
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    cur.execute("SELECT * from game_genre_vector_all g where g.appid=404730")
    arr = cur.fetchone()
arr

(404730,
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 b'\x00\x00\x00\x00',
 

In [27]:
tmp = [f"tag_{i}!=1" for i in range(1, 340)]
tmp1 = " AND ".join(tmp)
tmp1

'tag_1!=1 AND tag_2!=1 AND tag_3!=1 AND tag_4!=1 AND tag_5!=1 AND tag_6!=1 AND tag_7!=1 AND tag_8!=1 AND tag_9!=1 AND tag_10!=1 AND tag_11!=1 AND tag_12!=1 AND tag_13!=1 AND tag_14!=1 AND tag_15!=1 AND tag_16!=1 AND tag_17!=1 AND tag_18!=1 AND tag_19!=1 AND tag_20!=1 AND tag_21!=1 AND tag_22!=1 AND tag_23!=1 AND tag_24!=1 AND tag_25!=1 AND tag_26!=1 AND tag_27!=1 AND tag_28!=1 AND tag_29!=1 AND tag_30!=1 AND tag_31!=1 AND tag_32!=1 AND tag_33!=1 AND tag_34!=1 AND tag_35!=1 AND tag_36!=1 AND tag_37!=1 AND tag_38!=1 AND tag_39!=1 AND tag_40!=1 AND tag_41!=1 AND tag_42!=1 AND tag_43!=1 AND tag_44!=1 AND tag_45!=1 AND tag_46!=1 AND tag_47!=1 AND tag_48!=1 AND tag_49!=1 AND tag_50!=1 AND tag_51!=1 AND tag_52!=1 AND tag_53!=1 AND tag_54!=1 AND tag_55!=1 AND tag_56!=1 AND tag_57!=1 AND tag_58!=1 AND tag_59!=1 AND tag_60!=1 AND tag_61!=1 AND tag_62!=1 AND tag_63!=1 AND tag_64!=1 AND tag_65!=1 AND tag_66!=1 AND tag_67!=1 AND tag_68!=1 AND tag_69!=1 AND tag_70!=1 AND tag_71!=1 AND tag_72!=1 AND 

In [29]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    cur.execute(f"SELECT appid FROM game_genre_vector_all where {tmp1}")
    arr = cur.fetchall()
arr

[]

In [28]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    cur.execute(f"DELETE FROM game_genre_vector_all where {tmp1}")
    conn.commit()
    

In [3]:
get_feature_vector_from_web(404730)

array([1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [4]:
np.array([0 for i in range(1, 340)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
a = [0 if i else 1 for i in range(340)]
a

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
import sqlite3

In [16]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    l = ",".join(["?"]*340)
    cur.execute(f"INSERT OR IGNORE INTO game_genre_vector_all VALUES ({l})", a)
    cur.execute(f"SELECT * FROM game_genre_vector_all g where g.appid=?", [a[0]])
    arr = cur.fetchall()
    conn.rollback()
arr

[(1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [15]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    cur.execute(f"DELETE FROM game_genre_vector_all where appid=1")
    conn.commit()
    

In [17]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    cur.execute(f"SELECT * FROM game_genre_vector_all g where g.appid=?", [a[0]])
    arr = cur.fetchall()
    conn.rollback()
arr

[]

In [2]:
df_final = pd.read_csv("games_final.csv", index_col=0)
df_final

,name,release_date,ccu,weighted_rating
appid,,,,
10,Counter-Strike,2000-11-01,15514,97.486678
20,Team Fortress Classic,1999-04-01,103,86.125120
30,Day of Defeat,2003-05-01,141,89.625277
40,Deathmatch Classic,2001-06-01,12,81.580127
50,Half-Life: Opposing Force,1999-11-01,218,94.892334
...,...,...,...,...
2208920,Assassin's Creed Valhalla,2022-12-06,9898,61.582820
2209680,Beach Invasion 1944,2022-12-02,15,82.724132
2216360,Disaster Band,2022-12-20,68,83.903285


In [5]:
df_final[df_final.release_date > "2020"]

,name,release_date,ccu,weighted_rating
appid,,,,
60,Ricochet,2022-01-11,11,81.850558
1313,SiN: Gold,2020-03-18,3,84.020928
12210,Grand Theft Auto IV: The Complete Edition,2020-03-24,2590,78.480827
212410,Inversion,2021-02-18,2,68.895307
234820,Driver Fusion - The Best Driver & Device Solution,2020-12-24,31,74.936419
...,...,...,...,...
2208920,Assassin's Creed Valhalla,2022-12-06,9898,61.582820
2209680,Beach Invasion 1944,2022-12-02,15,82.724132
2216360,Disaster Band,2022-12-20,68,83.903285


In [1]:
from flask_app.views.analysis_views import get_user_data, get_owned_games, store_user_data

In [2]:
user = get_user_data("line1029")
data = get_owned_games(user)
user

In [3]:
user.id

'76561198140200441'

In [4]:
data

{'game_count': 806,
 'games': [{'appid': 4000,
   'playtime_forever': 0,
   'playtime_windows_forever': 0,
   'playtime_mac_forever': 0,
   'playtime_linux_forever': 0,
   'rtime_last_played': 0},
  {'appid': 2520,
   'playtime_forever': 0,
   'playtime_windows_forever': 0,
   'playtime_mac_forever': 0,
   'playtime_linux_forever': 0,
   'rtime_last_played': 0},
  {'appid': 2525,
   'playtime_forever': 0,
   'playtime_windows_forever': 0,
   'playtime_mac_forever': 0,
   'playtime_linux_forever': 0,
   'rtime_last_played': 0},
  {'appid': 4540,
   'playtime_forever': 0,
   'playtime_windows_forever': 0,
   'playtime_mac_forever': 0,
   'playtime_linux_forever': 0,
   'rtime_last_played': 0},
  {'appid': 4550,
   'playtime_forever': 0,
   'playtime_windows_forever': 0,
   'playtime_mac_forever': 0,
   'playtime_linux_forever': 0,
   'rtime_last_played': 0},
  {'appid': 475150,
   'playtime_forever': 0,
   'playtime_windows_forever': 0,
   'playtime_mac_forever': 0,
   'playtime_linux_fo

In [6]:
import time
start = time.time()
store_user_data(user, data)
print("time:", time.time() - start)

         1    2    3    4    5    6    7    8    9    10   ...  330  331  332  \
0                                                          ...                  
10         0    1    0    0    0    1    0    0    1    0  ...    0    0    0   
20         0    1    0    0    0    0    0    0    1    0  ...    0    0    0   
30         0    1    0    1    0    1    0    0    1    0  ...    0    0    0   
40         0    1    0    0    0    0    0    0    1    0  ...    0    0    0   
50         0    1    1    1    0    0    0    0    0    1  ...    0    0    0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
1328670    0    1    0    1    0    0    0    1    0    1  ...    0    0    0   
1366540    1    0    0    1    1    1    1    0    0    0  ...    0    0    0   
1426210    0    1    1    0    0    0    0    0    1    0  ...    0    0    0   
1449560    0    1    0    0    0    0    0    0    0    0  ...    0    0    0   
1462570    1    1    1    1 

In [6]:
appids = [4000, 2520, 2525, 4540, 4550, 475150, 2500, 11200, 17470, 17410, 35700, 21980, 20900, 22370, 50620, 462780, 31280, 31290, 94500, 94510, 94520, 94530, 47780, 42910, 40800, 44340, 400, 620, 20920, 31270, 8980, 729040, 28050, 107100, 3830, 201830, 25000, 2820, 201310, 94590, 67370, 22380, 64000, 98400, 201700, 63380, 207610, 7670, 409710, 200260, 212680, 204180, 108800, 12750, 11590, 12830, 12810, 12770, 50300, 206440, 35140, 208500, 8330, 8340, 8350, 8360, 8370, 8380, 94600, 94610, 94620, 8200, 8210, 8220, 8240, 8250, 8260, 8270, 8280, 8290, 8300, 31220, 31230, 31240, 31250, 31260, 8310, 8320, 204630, 11450, 12710, 220200, 219640, 34030, 203680, 218410, 3700, 223730, 17460, 24980, 34330, 227220, 231200, 204240, 203160, 8870, 34870, 44350, 210770, 222480, 3910, 224760, 234650, 234710, 236090, 206190, 231160, 35720, 242760, 242820, 242920, 243160, 10, 20, 30, 40, 50, 60, 70, 80, 100, 130, 220, 240, 280, 300, 320, 340, 360, 380, 420, 500, 550, 730, 243950, 45740, 247370, 225080, 239030, 214560, 860950, 239070, 249590, 249650, 209000, 249990, 250050, 250180, 250260, 250320, 251060, 251430, 251470, 251570, 251710, 251990, 252490, 700580, 252670, 252950, 253230, 208610, 245170, 254320, 205100, 255220, 245620, 219990, 209370, 239700, 257350, 257850, 250400, 49520, 258910, 221910, 260230, 238430, 261030, 261570, 261640, 262060, 263060, 263300, 263980, 264710, 108600, 7200, 228760, 232910, 243360, 227300, 224460, 265890, 266130, 211820, 367540, 268870, 269710, 269790, 270150, 8930, 271590, 271900, 237990, 65780, 224480, 274170, 274190, 222880, 200510, 277700, 277890, 278100, 278360, 280200, 281200, 281610, 281640, 281990, 246620, 282100, 282140, 284240, 284850, 285820, 286570, 287260, 287340, 287700, 287980, 288160, 233290, 290300, 290770, 291010, 292500, 292600, 292910, 293780, 294100, 201810, 256290, 296870, 204360, 238460, 237930, 300380, 300550, 300570, 300580, 203730, 239430, 286690, 287390, 301640, 303210, 244160, 304410, 304430, 238090, 306040, 26800, 306660, 308040, 308060, 310560, 237850, 311340, 311690, 312530, 312610, 312840, 313740, 314020, 265590, 247080, 314790, 314830, 255070, 315430, 315670, 214830, 240970, 226580, 253510, 318600, 207170, 251730, 319560, 319910, 320040, 238010, 320340, 41900, 240760, 404730, 268050, 321360, 322190, 233610, 322500, 690040, 289930, 220440, 323190, 242640, 323580, 302510, 323850, 307690, 238320, 325090, 326180, 326520, 327510, 44360, 211420, 289650, 330830, 330840, 331190, 331870, 298110, 305620, 332500, 234630, 282070, 249050, 334230, 276810, 334940, 335300, 222420, 337850, 337960, 257510, 290790, 339200, 319630, 304650, 339800, 340000, 340490, 341800, 314660, 342380, 342620, 343710, 343800, 343860, 345090, 222440, 346940, 294440, 347160, 348250, 298630, 350640, 350970, 351920, 291650, 319140, 310890, 353640, 21690, 356040, 356050, 8850, 409720, 235540, 357780, 359100, 360150, 232090, 361300, 361990, 303390, 362680, 344410, 362890, 248610, 363440, 363490, 363600, 364640, 329460, 365450, 366230, 366250, 366260, 366270, 366280, 366320, 367450, 367500, 367520, 238370, 397080, 368370, 368420, 368730, 369990, 219740, 370130, 370360, 370910, 322330, 269670, 226860, 373480, 373930, 373990, 375200, 375820, 267750, 365160, 376870, 377360, 377680, 236870, 381320, 383580, 384110, 384190, 345180, 384550, 384630, 386070, 386750, 290340, 387290, 295790, 388410, 388750, 234140, 391540, 392410, 393420, 65980, 394690, 396350, 396710, 397340, 397460, 391260, 397500, 407230, 331600, 398850, 399430, 400110, 400170, 400180, 214950, 10500, 345260, 4700, 211160, 345240, 400450, 400630, 401810, 402020, 402180, 393380, 774941, 403970, 404540, 405640, 405650, 405710, 406150, 407330, 407810, 407980, 394510, 410820, 410900, 413150, 272890, 414700, 416600, 416680, 416770, 307670, 311210, 417860, 418060, 418240, 418360, 294000, 420290, 420360, 208650, 421050, 421700, 421740, 423230, 423580, 424840, 425580, 426000, 270880, 428550, 428750, 429570, 430960, 431960, 432260, 432620, 391220, 433340, 433850, 439700, 434460, 434650, 434660, 435400, 268500, 437390, 437630, 438490, 460870, 385730, 438790, 310380, 364360, 442780, 374320, 443810, 445980, 446020, 446840, 447780, 450390, 275850, 452570, 453910, 368390, 456750, 314710, 457140, 458680, 271240, 384250, 464060, 464920, 465760, 465840, 465870, 466560, 379720, 425220, 255710, 470260, 470780, 471010, 473530, 473690, 274500, 474960, 475190, 327030, 394360, 383150, 381120, 485000, 486990, 487350, 355800, 488730, 489140, 489520, 361230, 491950, 391040, 493540, 494720, 499520, 499890, 273350, 501300, 501320, 448510, 503560, 385760, 504460, 505730, 391720, 508530, 508790, 510620, 368230, 496300, 512230, 514900, 517000, 360430, 289070, 520600, 312660, 521500, 522210, 522240, 523780, 366640, 292030, 496730, 526870, 40980, 530020, 530320, 534290, 534550, 535850, 437570, 115800, 418370, 539670, 540840, 543460, 545270, 544330, 546560, 337340, 493340, 552590, 553960, 555610, 507490, 557600, 558420, 429660, 564750, 568220, 570460, 573170, 574090, 575510, 578080, 579760, 581270, 581520, 582550, 584980, 285190, 590560, 595140, 600370, 600660, 601430, 603800, 606150, 378540, 383980, 209080, 608800, 609320, 609490, 612390, 614090, 617830, 619700, 620980, 623940, 627620, 629770, 632000, 632470, 633130, 1304780, 633460, 374040, 635200, 646270, 646570, 403640, 614570, 278910, 650220, 655700, 656680, 658850, 20500, 667720, 670080, 493900, 674520, 677160, 678850, 680360, 683320, 686600, 230190, 414340, 747350, 535480, 40700, 691830, 641990, 696480, 584400, 698780, 240720, 635320, 709440, 709820, 435150, 611500, 719750, 306130, 731490, 732810, 736260, 743450, 476600, 476620, 753640, 760060, 379430, 589360, 782330, 686260, 802450, 637090, 812140, 337000, 826600, 552500, 840610, 570940, 440900, 931180, 858260, 542050, 862480, 718670, 882100, 356190, 897030, 638970, 673880, 412020, 1449560, 942970, 945360, 951940, 972660, 703880, 977950, 223850, 225540, 986800, 996580, 1004610, 752590, 1055540, 418460, 737230, 1070710, 1091500, 1101360, 1104660, 976730, 1150950, 485030, 1085660, 1172380, 1174180, 203770, 582010, 1145360, 1213210, 1222690, 1222700, 1222730, 1225580, 1225590, 1233570, 1238060, 1238000, 1237970, 1237950, 1292940, 1328670, 1366540, 1190460, 1426210, 1024650, 1462570, 1580970, 231430, 1180660, 17300, 460250]

In [17]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    vector = []
    for appid in appids:
        cur.execute("select * from game_genre_vector_all where appid=?;",(appid,))
        
        vector.append(cur.fetchone())
print(vector)

[(4000, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
len(vector)

57581

In [7]:
str(tuple(appids))

'(4000, 2520, 2525, 4540, 4550, 475150, 2500, 11200, 17470, 17410, 35700, 21980, 20900, 22370, 50620, 462780, 31280, 31290, 94500, 94510, 94520, 94530, 47780, 42910, 40800, 44340, 400, 620, 20920, 31270, 8980, 729040, 28050, 107100, 3830, 201830, 25000, 2820, 201310, 94590, 67370, 22380, 64000, 98400, 201700, 63380, 207610, 7670, 409710, 200260, 212680, 204180, 108800, 12750, 11590, 12830, 12810, 12770, 50300, 206440, 35140, 208500, 8330, 8340, 8350, 8360, 8370, 8380, 94600, 94610, 94620, 8200, 8210, 8220, 8240, 8250, 8260, 8270, 8280, 8290, 8300, 31220, 31230, 31240, 31250, 31260, 8310, 8320, 204630, 11450, 12710, 220200, 219640, 34030, 203680, 218410, 3700, 223730, 17460, 24980, 34330, 227220, 231200, 204240, 203160, 8870, 34870, 44350, 210770, 222480, 3910, 224760, 234650, 234710, 236090, 206190, 231160, 35720, 242760, 242820, 242920, 243160, 10, 20, 30, 40, 50, 60, 70, 80, 100, 130, 220, 240, 280, 300, 320, 340, 360, 380, 420, 500, 550, 730, 243950, 45740, 247370, 225080, 239030, 2

In [9]:
import sqlite3
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    total_vector = []
    l = str(tuple(appids))
    cur.execute(f"select * from game_genre_vector_all where appid in {l}")
    vector = cur.fetchall()
    diff = set(appids).difference(set(i[0] for i in vector))

In [11]:
print(vector)

[(10, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [18]:
diff

{28050, 208610, 367540, 397080, 700580, 737230, 774941, 931180, 1580970}

In [20]:
for i in diff:
    print(i)

208610
700580
1580970
931180
737230
28050
367540
397080
774941


In [15]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    ll = str(tuple(diff))
    cur.execute(f"select * from game_genre_vector_all where appid in {ll}")
    vv = cur.fetchall()
vv

[]

In [22]:
with sqlite3.connect("flask_app.db") as conn:
    cur = conn.cursor()
    cur.execute("select * from game_genre_vector_all where appid in ()")
    vv = cur.fetchall()
vv

[]

In [21]:
str(tuple([]))

'()'

In [23]:
dict((i,i) for i in range(5))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

In [10]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
a = df.sum()
df.loc[2] *= 2
b = df.sum()

In [11]:
a

A     6
B    15
dtype: int64

In [12]:
b

A     9
B    21
dtype: int64

In [13]:
c = df.loc[2]

In [14]:
c

A     6
B    12
Name: 2, dtype: int64

In [15]:
df.loc[2] *= 2
c

A    12
B    24
Name: 2, dtype: int64

In [18]:
from datetime import datetime
datetime.now()

datetime.datetime(2023, 1, 3, 16, 33, 6, 829579)

In [20]:
sum([])

0